## 准备数据

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=(
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

docs = text_splitter.split_documents(documents)

docs[1]

USER_AGENT environment variable not set, consider setting it to identify your requests.


Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a 

## 实例化Milvus的向量数据库，该数据库会将文档加载到Milvs向量数据库并在后台构建索引

In [2]:
import sys
from langchain_milvus import Milvus
sys.path.append("..")
from llm_utils import qwen_embeddings,llm,openai_embedding

docs = docs[:10]  # 限制文档数量，避免批量过大报错
vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=openai_embedding,
    connection_args={
        "uri": "http://localhost:19530",  # http://localhost:19530
    },
    collection_name="langchain_example",
    drop_old=True,  # 删除现有集合以解决维度不匹配问题
)

2025-08-26 17:50:43,263 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)


## 检索

In [3]:
query = "What is self-reflection of an AI Agent?"
vectorstore.similarity_search(query, k=1)     # 实际上后续用LCEL 实现 用构建好的 retrieval |  format_docs 来拿到模型期望的<context>字符串格式类型

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'pk': 460375432065191637}, page_content='Self-Reflection#\nSelf-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.\nReAct (Yao et al. 2023) integrates reasoning and acting within LLM by extending the action space to be a combination of task-specific discrete actions and the language space. The former enables LLM to interact with the environment (e.g. use Wikipedia search API), while the latter prompting LLM to generate reasoning traces in natural language.\nThe ReAct prompt template incorporates explicit steps for LLM to think, roughly formatted as:\nThought: ...\nAction: ...\nObservation: ...\n... (Repeated many times)')]

In [4]:
# Define the prompt template for generating AI responses
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# 您是一个人工智能助手，在可能的情况下会基于事实和统计数据来回答问题。
# 请利用以下信息为<question>标签内的问题提供简洁回答。
# 若不知道答案，请直接说明不清楚，不要试图编造答案。
PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
prompt = PromptTemplate(
    template = PROMPT_TEMPLATE,
    input_variables = ["context", "question"]
)

# 构建检索器retrieval，便于集成到LCEL
retrieval = vectorstore.as_retriever()

# 定于检索器输出处理函数
def format_docs(docs):
    """Format the retrieved documents into a single string."""
    return "\n\n".join(doc.page_content for doc in docs)

## 构建LECL处理逻辑链

In [6]:
rag_chain = (
    {"context": retrieval |  format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# rag_chain.get_graph().print_ascii()

# Invoke the RAG chain with a specific question and retrieve the response
# res = rag_chain.invoke(query)
# print(res)
# stream输出
for chunk in rag_chain.stream(query):
    print(chunk, end="|", flush=True) 

|Self|-ref|lection| of| an| AI| agent| is| a| process| that| allows| the| agent| to| improve| its| performance| by| analyzing| past| actions| and| correcting| mistakes|.| It| is| vital| for| iterative| improvement|,| particularly| in| environments| where| trial| and| error| is| necessary|.| In| the| Reflex|ion| framework|,| self|-ref|lection| involves| evaluating| the| efficiency| of| a| trajectory| and| identifying| errors| such| as| ineff|iciencies| and| halluc|inations|.| 

|For| instance|,| halluc|ination| appears| to| be| a| more| prevalent| issue| than| inefficient| planning| in| specific| tasks| like| Alf|World|,| according| to| experiments|.| Self|-ref|lection| enhances| reasoning| skills| by| integrating| examples| of| failed| trajectories| and| ideal| reflections| into| the| agent|'s| memory| for| ongoing| learning|.| By| doing| this|,| AI| agents| can| refine| their| strategies| and| decision|-making| processes| over| time|.||